
<div style = " margin-left: auto;
    margin: auto;
    display: block;
    background-color: lightblue;
    width: 50%
    Padding: 15px">
<img src = "ForSale.jpg" style="display: block;margin-left: auto;
    margin-right: auto;
    width: 50%;
    padding: 15px">

<p style = "text-align: center; 
    color:blue;
    font-size: 2.5em;
    font-Weight: bold;
    font-family: serif">Kaggle House Prices Challenge</p>
<p style = "text-align: center;
    color: red;
    font-size: 1.5em;
    font-Weight: bold; 
    font-family: serif;">Using Linear Regression</p>

<p style = "text-align: center">Brandon Hart</p>
<p style = "text-align: center; 
    font-weight: bold;
    padding-bottom: 15px">dataNinja</p>
</div>


In [ ]:
# load Libraries

# Data Objects
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt
from pylab import rcParams

#ML Algorythms and scoring
from scipy import stats
from sklearn.metrics import mean_squared_error as MSE

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.linear_model import Lasso



In [ ]:
#Load Data
trainingData = pd.read_csv("train.csv")
competitionData = pd.read_csv("test.csv")


In [ ]:
#Set Plotting parameters
%matplotlib inline
rcParams['figure.figsize'] = 10,4

In [ ]:
print("Training Data Shape (Rows,Columns):",trainingData.shape)
print("Competition Data Shape (Rows,Columns):", competitionData.shape)


In [ ]:
display(trainingData.head())
display(competitionData.head())


In [ ]:
#Lets see all columns
pd.options.display.max_columns = None
display(trainingData.head())
display(competitionData.head())



<h1> Analyzing the Data Set </h1>
<ul>
<li> 79 features!  Thats a lot to think about!  </li>
<li> A few NaN (Not Numbers): We have to think about how we deal with those</li>
<li> Discrete features like in Street, LotShape, LandContour, et, we must find a way to make these continuous for Regression</li>
<li> Data looks like a normal ditribution but is skewed</li>
<li> ID Column needs to be pulled out </li>
</ul>
<p> The goal is to transform this data into a form where regression can be done. </p>

In [ ]:

# Do not want regression algorythm to consider the feature 'Id' so will remove them and replace them as the dataframe index:
# This will also make it available to use as a primary key for X and y sets

trainingData.set_index('Id',inplace = True)
competitionData.set_index('Id',inplace = True)

display(trainingData.head())
display(competitionData.head())







In [ ]:
# For discrete items, replace nan with "Missing", for continuous items fill with 0

trainingDataColumns = list(trainingData) 
  
for c in trainingDataColumns: 
    if trainingData[c].dtype == 'O':
        trainingData[c].fillna(value = 'Missing', inplace = True)
    else :
        trainingData[c].fillna(0, inplace = True)
            

# Do the same for competition data

competitionDataColumns = list(competitionData) 
  
for f in competitionDataColumns: 
    if competitionData[f].dtype == 'O':
        competitionData[f].fillna(value = 'Missing', inplace = True)
    else :
        competitionData[f].fillna(0, inplace = True)

display(trainingData.head())
display(competitionData.head())

In [ ]:
# Tranform discrete values to columns with 1 and 0s
trainingData = pd.get_dummies(trainingData)

# Do the same for competition data
competitionData = pd.get_dummies(competitionData)


display(trainingData.head())
display(competitionData.head())


In [ ]:
print("Training Data Shape (Rows,Columns):",trainingData.shape)
print("Competition Data Shape (Rows,Columns):", competitionData.shape)



In [ ]:
#Ooops!!!  There is a differece between features in the the training data set and the competition data set
# We will try dropping the features that are not present in both sets

sp = trainingData['SalePrice']

missingFeatures = list(set(trainingData.columns.values) - set(competitionData.columns.values))
trainingData = trainingData.drop(missingFeatures,axis=1)

missingFeatures = list(set(competitionData.columns.values) - set(trainingData.columns.values))
competitionData = competitionData.drop(missingFeatures,axis=1)

print("Training Data Shape (Rows,Columns):",trainingData.shape)
print("Competition Data Shape (Rows,Columns):", competitionData.shape)


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(trainingData, sp, random_state=0)

In [ ]:
# Lasso is a form of linear regression that restricts coefficients to be close to zero or exactly zero.
# This acts as a form of automatic feature selection
# alpha is how strongly the coefficients are pushed to zero
#  I performed a loop on alpha to get the one that returned the highest test scores, removed for faster performance

myModel = Lasso(alpha=298.4).fit(X_train,y_train)
print ("Train score: ",myModel.score(X_train,y_train),"\nTest score: ", myModel.score(X_test,y_test))
print ("Number of features used: {}".format(np.sum(myModel.coef_ != 0)))

In [ ]:
#Examine Top Ten Features

featureDF = pd.DataFrame(trainingData.columns.values,columns=['Features'])
featureDF['w'] = myModel.coef_
featureDF['Abs(w)'] = featureDF['w'].abs()
featureDF = featureDF.sort_values(by=['Abs(w)'], ascending=False)
featuresUsed = featureDF.head(10)

display(featuresUsed[['Features','w']])

In [ ]:
# Graph Top Ten Features using bar graph
colors = ['tab:blue', 'tab:orange', 'tab:green', 'tab:red', 'tab:purple', 'tab:brown', 'tab:pink', 'tab:gray', 'tab:olive', 'tab:cyan']
plt.title('Top 10 Features')
plt.xticks(rotation=45)
plt.ylabel('Weights')
plt.xlabel('Features')
plt.yticks(np.arange(-65000, 65000, step=20000))
featureList = plt.bar(featuresUsed['Features'],featuresUsed['w'],color=colors)
plt.show(block=False)

In [ ]:
#Prepare a datafram called results for review

results = pd.DataFrame(myModel.predict(X_test), columns=['Prediction'], index = X_test.index)
results['Prediction'] = results['Prediction'].round()
results['Actual'] = y_test
results['$Variance'] = results['Prediction'] - results['Actual']
results['%Variance'] = results['$Variance'] / results['Actual']

In [ ]:
# Review First 10 results & RMSE

display(results.head(10))
mse = MSE(np.log(results.Actual),np.log(results.Prediction))
rmse = mse**(1/2)
display("RMSE: ", rmse)

In [ ]:
fig = plt.figure()
ax1 = fig.add_subplot(111)

ax1.scatter(results.index, results['Prediction'], s=10, c='b', marker="s", label='Prediction')
ax1.scatter(results.index, results['Actual'], s=10, c='r', marker="o", label='Actual')
plt.legend(loc='upper left');
plt.show()

In [ ]:
submission = pd.DataFrame(myModel.predict(competitionData), columns=['SalePrice'], index = competitionData.index)

display(submission.head())

In [ ]:
submission.to_csv("submission_bhart.csv")